In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
%matplotlib inline

In [ ]:
Train_df = pd.read_csv(r"C:\Users\Shawlock\Downloads\all\train.csv")
Test_df = pd.read_csv(r"C:\Users\Shawlock\Downloads\all\test.csv")

In [ ]:
Train_df.head(5)

In [ ]:
Test_df.head(5)

In [ ]:
train_ID = Train_df['Id']
test_ID = Test_df['Id']

# Now drop the 'Id' colum since it's unnecessary for the prediction process
Train_df.drop("Id", axis = 1, inplace = True)
Test_df.drop("Id", axis = 1, inplace = True)
Train_df.shape


In [ ]:
Test_df.shape

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x=Train_df["GrLivArea"], y= Train_df["SalePrice"])
plt.xlabel("GrLivArea", fontsize = 13)

plt.ylabel("SalePrice", fontsize = 13)

In [ ]:
Train_df = Train_df.drop(Train_df[(Train_df["GrLivArea"]>4000) & (Train_df["SalePrice"]<300000)].index)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x=Train_df["GrLivArea"], y= Train_df["SalePrice"])
plt.xlabel("GrLivArea", fontsize = 13)

plt.ylabel("SalePrice", fontsize = 13)

In [ ]:
#Now we will check our target variable for the skewness
(mu, sigma) = norm.fit(Train_df['SalePrice'])

# 1. Plot Sale Price
sns.distplot(Train_df['SalePrice'] , fit=norm);
plt.ylabel('Frequency')
plt.title('SalePrice distribution')
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')

# Get the fitted parameters used by the function
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

In [ ]:
# 2. Plot SalePrice as a QQPlot
fig = plt.figure()
res = stats.probplot(Train_df['SalePrice'], plot=plt)
plt.show()

In [ ]:
#let's try the log tranformation to make it normal
Train_df["SalePrice"] = np.log1p(Train_df["SalePrice"])


In [ ]:
# 1. Plot Sale Price
sns.distplot(Train_df['SalePrice'] , fit=norm);
plt.ylabel('Frequency')
plt.title('SalePrice distribution')
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(Train_df['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))


In [ ]:
# 2. Plot SalePrice as a QQPlot
fig = plt.figure()
res = stats.probplot(Train_df['SalePrice'], plot=plt)
plt.show()

In [ ]:
# Saving train & test shapes
ntrain = Train_df.shape[0]
ntest = Test_df.shape[0]

# Creating y_train variable
y_train = Train_df.SalePrice.values
Dataset = pd.concat((Train_df, Test_df)).reset_index(drop=True)
Dataset.drop(['SalePrice'], axis=1, inplace=True)
print("Dataset size is : {}".format(Dataset.shape))

In [ ]:
Dataset.info()

In [ ]:
#For the further processing, let's separate numeric and categorical data
numeric_data = Train_df.select_dtypes(include = [np.number])
categoric_data = Train_df.select_dtypes(exclude=[np.number])
print("there are {} numeric and {} categoric features in the dataset".format(numeric_data.shape[1], categoric_data.shape[1]))

In [ ]:
#Corelation for the numeric data
corr = numeric_data.corr()
sns.heatmap(corr)

In [ ]:
#enlarged heatmap with correlations
k=10
cols = corr.nlargest(k, "SalePrice")["SalePrice"].index
cm = np.corrcoef(Train_df[cols].values.T)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
print(corr["SalePrice"].sort_values(ascending=False)[:15],'\n')#top 15 values
print("------------------------------------------------------------")
print(corr["SalePrice"].sort_values(ascending=False)[-5:])#last 15 values

In [ ]:
Train_df["LotShape"].unique()

In [ ]:
#let's check the mean price per quality and plot it
pivot = Train_df.pivot_table(index="OverallQual" , values= "SalePrice", aggfunc=np.median)
pivot

In [ ]:
pivot.plot(kind="bar", color='orange')

In [ ]:



categoric_data.describe()

In [ ]:
pivot2 = Train_df.pivot_table(index="GarageType" , values= "SalePrice", aggfunc=np.median)
pivot2

In [ ]:
pivot2.plot(kind="bar", color='violet')

In [ ]:
all_data = (Dataset.isnull().sum() / len(Dataset)) * 100 #Total missing values in the dataset
all_data = all_data.drop(all_data[all_data == 0].index).sort_values(ascending=False)[:30] #Excluding values with "0" as an entry

In [ ]:
#percent missing data
f, ax= plt.subplots(figsize=(15,12))
plt.xticks(rotation="90")
sns.barplot(x=all_data.index, y=all_data)
plt.xlabel("Features", fontsize=15)
plt.ylabel("Percent of missing value", fontsize=15)
plt.title("Percent missing data by featue", fontsize=15)

In [ ]:
Dataset["PoolQC"] = Dataset["PoolQC"].fillna("None")

In [ ]:
Dataset["MiscFeature"] = Dataset["MiscFeature"].fillna("None")

In [ ]:
Dataset["Alley"] = Dataset["Alley"].fillna("None")

In [ ]:
Dataset["Fence"] = Dataset["Fence"].fillna("None")

In [ ]:
Dataset["FireplaceQu"] = Dataset["FireplaceQu"].fillna("None")

In [ ]:
Dataset["LotFrontage"] = Dataset.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

In [ ]:
for col in ["GarageFinish", "GarageQual", "GarageCond", "GarageType"]:
    Dataset[col] = Dataset[col].fillna("None")

In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    Dataset[col] = Dataset[col].fillna(0)

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    Dataset[col] = Dataset[col].fillna(0)

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    Dataset[col] = Dataset[col].fillna('None')

In [ ]:
Dataset["MasVnrType"] = Dataset["MasVnrType"].fillna("None")
Dataset["MasVnrArea"] = Dataset["MasVnrArea"].fillna(0)

In [ ]:
Dataset['MSZoning'] = Dataset['MSZoning'].fillna(Dataset['MSZoning'].mode()[0])

In [ ]:
Dataset = Dataset.drop(['Utilities'], axis=1)

In [ ]:
Dataset["Functional"] = Dataset["Functional"].fillna("Typ")

In [ ]:
Dataset['Electrical'] = Dataset['Electrical'].fillna(Dataset['Electrical'].mode()[0])

In [ ]:
Dataset['KitchenQual'] = Dataset['KitchenQual'].fillna(Dataset['KitchenQual'].mode()[0])

In [ ]:
Dataset['Exterior1st'] = Dataset['Exterior1st'].fillna(Dataset['Exterior1st'].mode()[0])
Dataset['Exterior2nd'] = Dataset['Exterior2nd'].fillna(Dataset['Exterior2nd'].mode()[0])

In [ ]:
Dataset['SaleType'] = Dataset['SaleType'].fillna(Dataset['SaleType'].mode()[0])

In [ ]:
Dataset['MSSubClass'] = Dataset['MSSubClass'].fillna("None")

In [ ]:
#Check remaining missing values if any 
all_data_na = (Dataset.isnull().sum() / len(Dataset)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

In [ ]:
# Converting those variables which should be categorical, rather than numeric
for col in ('MSSubClass', 'OverallCond', 'YrSold', 'MoSold'):
    Dataset[col] = Dataset[col].astype(str)
    
Dataset.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(Dataset[c].values)) 
    Dataset[c] = lbl.transform(list(Dataset[c].values))

# shape        
print('Shape all_data: {}'.format(Dataset.shape))

In [ ]:
# Adding total sqfootage feature as new feature
Dataset['TotalSF'] = Dataset['TotalBsmtSF'] + Dataset['1stFlrSF'] + Dataset['2ndFlrSF']

In [ ]:
from scipy.stats import skew
# Applying a log(1+x) transformation to all skewed numeric features
numeric_feats = Dataset.dtypes[Dataset.dtypes != "object"].index

# Compute skewness
skewed_feats = Dataset[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(15)

In [ ]:
# Check on number of skewed features above 75% threshold
skewness = skewness[abs(skewness) > 0.75]
print("Total number of features requiring a fix for skewness is: {}".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feature in skewed_features:
    Dataset[feature] = boxcox1p(Dataset[feature], lam)


In [ ]:
Dataset = pd.get_dummies(Dataset)
print(Dataset.shape)